In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
Author: Gregor Pfalz
github: GPawi

Plans for v2: Automatically detect based on LabID, from which lab the age determination comes
"""

'\nAuthor: Gregor Pfalz\ngithub: GPawi\n\nPlans for v2: Automatically detect based on LabID, from which lab the age determination comes\n'

In [2]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import os
#import sys
import sqlalchemy
#import datetime
from sqlalchemy.exc import IntegrityError
from xlrd import XLRDError
from tkinter import Tk
from tkinter.filedialog import askopenfilename

In [3]:
class connection_db(object):
    def __init__(self, filename = None):
        self.filename = filename
        #sheets = ['DatasetDescription',
        #               'Organic',
        #               'GrainSize',
        #               'Element',
        #               'Mineral',
        #               'Diatom',
        #               'Chironomid',
        #               'Pollen',
        #               'Age']
                
    def select_data(self):
        filename = self.filename
        if filename == None: 
            root = Tk()
            root.withdraw()
            root.call('wm', 'attributes', '.', '-topmost', True)
            filename = askopenfilename()
            %gui tk
            xl = pd.ExcelFile(filename)
            self.input_dictionary = {}
            for sheet in xl.sheet_names:
                self.input_dictionary[f'{sheet}']= pd.read_excel(xl,sheet_name=sheet)
        else:
            xl = pd.ExcelFile(filename)
            self.input_dictionary = {}
            for sheet in xl.sheet_names:
                self.input_dictionary[f'{sheet}']= pd.read_excel(xl,sheet_name=sheet)
                
    def main_input(self):
        input_dictionary = self.input_dictionary
        self.input_main = input_dictionary['DatasetDescription']
        
    def coreid(self):
        input_main = self.input_main
        ##
        try:
            self.coreid = input_main[(input_main == 'Unique CoreID').any(1)].dropna(axis = 1).iloc[0,1]
        except IndexError:
            raise Exception('Please add a unique CoreID!')
        
    def retrieve_data(self): ## retrieve existing information from the database
        pass
    
    def scientist_input(self):
        input_main = self.input_main
        coreid = self.coreid
        ##
        ### PI ###
        self.scientist_columns = ['firstname','lastname','email','orcid']
        self.scientist = pd.DataFrame(columns = self.scientist_columns)
        try:
            self.firstname = input_main[(input_main == 'First Name Contributer').any(1)].dropna(axis = 1).iloc[0,1]
            self.scientist.at[0,'firstname'] = self.firstname
            self.lastname = input_main[(input_main == 'Last Name Contributer').any(1)].dropna(axis = 1).iloc[0,1]
            self.scientist.at[0,'lastname'] = self.lastname
        except IndexError:
            raise Exception('Please add the first and last name of the main contributer!')
        try:
            self.email = input_main[(input_main == 'Email').any(1)].dropna(axis = 1).iloc[0,1]
        except IndexError:
            print(f'No Email found for {self.firstname} {self.lastname} - will set NULL')
            self.email = None
        self.scientist.at[0,'email'] = self.email
        try:
            self.orcid = input_main[(input_main == 'ORCID').any(1)].dropna(axis = 1).iloc[0,1]
        except IndexError:
            print(f'No ORCID found for {self.firstname} {self.lastname} - will set NULL')
            self.orcid = None
        self.scientist.at[0,'orcid'] = self.orcid
        ### Additional Scientists ###
        self.start_index_addsci = input_main[(input_main == 'Additional involved scientist').any(1)].dropna(axis = 1).index[0]
        self.start_index_addsci = self.start_index_addsci + 1
        self.stop_index_addsci = input_main[(input_main == 'Additional involved scientist')].index.stop
        
        self.start_column_addsci = input_main[(input_main == 'Additional involved scientist').any(1)].dropna(axis = 1).columns[0]
        self.start_column_addsci = (input_main.columns.get_loc(self.start_column_addsci))
        self.stop_column_addsci = self.start_column_addsci + 4
        
        self.addsci = input_main.iloc[self.start_index_addsci:self.stop_index_addsci,self.start_column_addsci:self.stop_column_addsci]
        self.addsci.rename(columns=self.addsci.iloc[0], inplace = True)
        if len(self.addsci) > 1:
            self.addsci = self.addsci.drop(self.addsci.index[0])
            self.addsci.rename(columns = {'First Name':'firstname',
                                          'Last Name':'lastname',
                                          'Email':'email',
                                          'ORCID':'orcid'})
            self.scientist = self.scientist.append(self.addsci, ignore_index = True)
        else:
            print(f'No additional scientist found for {coreid}')
            pass
        
    def expedition_input(self):
        input_main = self.input_main
        ##
        self.expedition_columns = ['expeditionname','expeditionyear']
        self.expedition = pd.DataFrame(columns = self.expedition_columns)
        try:
            self.expeditionname = input_main[(input_main == 'Name of Field Campaign').any(1)].dropna(axis = 1).iloc[0,1]
            self.expedition.at[0,'expeditionname'] = self.expeditionname
            self.expeditionyear = input_main[(input_main == 'Year of Field Campaign').any(1)].dropna(axis = 1).iloc[0,1]
            self.expedition.at[0,'expeditionyear'] = self.expeditionyear
        except IndexError:
            raise Exception('Please add information about the expedition!')
            
    def lake_input(self):
        input_main = self.input_main
        ##
        self.lake_columns = ['sitename','country',
                             'lakedepth','lakeextent',
                             'catchmentarea','climatezone',
                             'vegetationzone','laketype']
        self.lake = pd.DataFrame(columns = self.lake_columns)
        try:
            self.sitename = input_main[(input_main == 'Site Name').any(1)].dropna(axis = 1).iloc[0,1]
            self.lake.at[0,'sitename'] = self.sitename
            self.country = input_main[(input_main == 'Country').any(1)].dropna(axis = 1).iloc[0,1]
            self.lake.at[0,'country'] = self.country
            self.lakedepth = input_main[(input_main == 'Lake Depth (m)').any(1)].dropna(axis = 1).iloc[0,1]
            self.lake.at[0,'lakedepth'] = self.lakedepth
            self.lakeextent = input_main[(input_main == 'Lake Extent (km²)').any(1)].dropna(axis = 1).iloc[0,1]
            self.lake.at[0,'lakeextent'] = self.lakeextent
            self.catchmentarea = input_main[(input_main == 'Catchment Area (km²)').any(1)].dropna(axis = 1).iloc[0,1]
            self.lake.at[0,'catchmentarea'] = self.catchmentarea
            self.climatezone = input_main[(input_main == 'Climate Zone').any(1)].dropna(axis = 1).iloc[0,1]
            self.lake.at[0,'climatezone'] = self.climatezone
            self.vegetationzone = input_main[(input_main == 'Vegetation Zone').any(1)].dropna(axis = 1).iloc[0,1]
            self.lake.at[0,'vegetationzone'] = self.vegetationzone
            self.laketype = input_main[(input_main == 'Lake Type').any(1)].dropna(axis = 1).iloc[0,1]
            self.lake.at[0,'laketype'] = self.laketype
        except IndexError:
            raise Exception('Please add more information about the lake!')
    
    def drilling_input(self):
        input_main = self.input_main
        coreid = self.coreid
        sitename = self.sitename
        expeditionname = self.expeditionname
        expeditionyear = self.expeditionyear
        ##
        self.drilling_columns = ['coreid',
                                 'latitude','longitude',
                                 'waterdepth','corelength',
                                 'drillingdevice','sitename',
                                 'expeditionname','expeditionyear']
        self.drilling = pd.DataFrame(columns = self.drilling_columns)
        try:
            self.drilling.at[0,'coreid'] = coreid
            self.latitude = input_main[(input_main == 'Latitude').any(1)].dropna(axis = 1).iloc[0,1]
            self.drilling.at[0,'latitude'] = self.latitude
            self.longitude = input_main[(input_main == 'Longitude').any(1)].dropna(axis = 1).iloc[0,1]
            self.drilling.at[0,'longitude'] = self.longitude
            self.waterdepth = input_main[(input_main == 'Water Depth (m)').any(1)].dropna(axis = 1).iloc[0,1]
            self.drilling.at[0,'waterdepth'] = self.waterdepth
            self.corelength = input_main[(input_main == 'Core Length (m)').any(1)].dropna(axis = 1).iloc[0,1]
            self.drilling.at[0,'corelength'] = self.corelength
            self.drillingdevice = input_main[(input_main == 'Drilling Device').any(1)].dropna(axis = 1).iloc[0,1]
            self.drilling.at[0,'drillingdevice'] = self.drillingdevice
            self.drilling.at[0,'sitename'] = sitename
            self.drilling.at[0,'expeditionname'] = expeditionname
            self.drilling.at[0,'expeditionyear'] = expeditionyear
        except IndexError:
            raise Exception('Please add more information about the lake!')
    
    def source_input(self):
        input_main = self.input_main
        ##
        self.start_index_source = input_main[(input_main == 'Sources').any(1)].dropna(axis = 1).index[0]
        self.start_index_source = self.start_index_source + 1 
        self.stop_index_source = input_main[(input_main == 'Age').any(1)].dropna(axis = 1).index[0]
        self.stop_index_source = self.stop_index_source + 1
        
        self.start_column_source = input_main[(input_main == 'Sources').any(1)].dropna(axis = 1).columns[0]
        self.start_column_source = (input_main.columns.get_loc(self.start_column_source))
        self.stop_column_source = self.start_column_source + 4
        
        self.source = input_main.iloc[self.start_index_source:self.stop_index_source, self.start_column_source:self.stop_column_source]
        self.source.rename(columns= self.source.iloc[0], inplace = True)
        self.source = self.source.drop(self.source.index[0])
        self.source = self.source.rename(columns = {'Entity':'entity',
                                                    'Repository':'repository',
                                                    'File Name':'filename',
                                                    'Accessible':'accessible'})
        self.source = self.source.dropna(axis = 0)
        
    def publication_input(self):
        input_main = self.input_main
        ##
        pass
        
    def organic_input(self):
        pass
    
    def grainsize_input(self):
        pass
    
    def element_input(self):
        pass
    
    def mineral_input(self):
        pass
    
    def diatom_input(self):
        pass
    
    def chironomid_input(self):
        pass
    
    def pollen_input(self):
        pass
    
    def age_input(self):
        pass
    
    def measurement_input(self):
        pass
    
    def data_prep(self):
        self.main_input()
        self.coreid()
        self.scientist_input()
        self.expedition_input()
        self.lake_input()
        self.drilling_input()
        self.source_input()
        self.publication_input()
        self.organic_input()
        self.grainsize_input()
        self.element_input()
        self.mineral_input()
        self.diatom_input()
        self.chironomid_input()
        self.pollen_input()
        self.age_input()
        self.measurement_input()
        
    def check_data(self):
        pass

In [7]:
core = connection_db()
#%timeit -n 1 -r 1 
core.select_data()
#%timeit -n 1 -r 1 
core.data_prep()

8.74 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
No ORCID found for Martin Melles - will set NULL
47.4 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [9]:
core.input_main

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,Essential information about the dataset,NaN,NaN,!Needs to be filled out before entered into th...,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,Unique CoreID,PG1111,NaN,NaN
5,NaN,-------------------------------------------,NaN,NaN,NaN
6,NaN,Name of Field Campaign,Norilsk/Taymyr,NaN,NaN
7,NaN,Year of Field Campaign,1993,NaN,NaN
8,NaN,-------------------------------------------,NaN,NaN,NaN
9,NaN,First Name Contributer,Martin,NaN,NaN
